In [ ]:
import os
from SpikeTrainUtils import SpikeTrainStats, SpikeTrainPlotter

In [ ]:
d = './results/'
dirs = [os.path.join(d, o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

sorted_dirs = []
for directory in dirs:
    split_d = directory.split(' ')
    sorted_dirs.append([split_d[1], split_d[3], directory])

sorted_dirs = sorted(sorted_dirs, key = lambda x: (int(x[0]), int(x[1])))

In [ ]:
sts = SpikeTrainStats()

results = []
debug_cnt = 10
cnt = 0
for directory in sorted_dirs:

    job_dir = os.listdir(directory[2])[-1]
    sub_directory = os.path.join(directory[2], job_dir)

    files = [os.path.join(sub_directory, o) for o in os.listdir(sub_directory) if os.path.isfile(os.path.join(sub_directory,o))]
    config_res = []
    for f in files:
        fname,ext = os.path.splitext(f)
        if ext != '.csv':
            continue
        fr, cvisi = sts.calculate(f)
        config_res.append([f.split(' ')[10],{'firing_rate':fr, 'cv_isi':cvisi}])
    results.append([directory,config_res])
    print results[-1]
    cnt += 1
    if cnt == debug_cnt:
        break
    

In [ ]:
from IPython.display import HTML, display

pop_types = ['exc', 'inh', 'ext']
pop_labels = {'exc':'Excitatory', 'inh':'Inhibitory', 'ext':'Injection'}

for pop_type in pop_types:
    slim_data = []
    for row in results:     
        for i in range(len(row[1])):

            if row[1][i][0] != pop_type:
                continue
            slim_data.append([row[0][0], row[0][1], row[1][i][1]['firing_rate'], row[1][i][1]['cv_isi']])
    print (pop_labels[pop_type] + " Population")
    table_head = '<table>'
    table_header = '<tr><th>G Exc</th><th>G Inh</th><th>Firing Rate</th><th>CV ISI</th></tr>'
    table_contents = '<tr>{}</tr>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in slim_data))
    table_tail = '</table>'
    html = table_head + table_header + table_contents + table_tail
    display(HTML(html))